# 

In [6]:
import qubovert as qv
import networkx as nx
from fastqaoa import Diagonals
from fastqaoa.ctypes.qaoa import apply_diagonals, apply_rx
from fastqaoa.ctypes.statevector import Statevector

import numpy as np

In [7]:
d_qubo = {"(0, 5)": "0.5", "(0, 16)": "0.5", "(0, 15)": "0.5", "(1, 3)": "0.5", "(1, 12)": "0.5", "(1, 7)": "0.5", "(2, 8)": "0.5", "(2, 26)": "0.5", "(2, 10)": "0.5", "(3, 8)": "0.5", "(3, 23)": "0.5", "(4, 19)": "0.5", "(4, 22)": "0.5", "(4, 25)": "0.5", "(5, 19)": "0.5", "(5, 21)": "0.5", "(6, 11)": "0.5", "(6, 14)": "0.5", "(6, 7)": "0.5", "(7, 23)": "0.5", "(8, 19)": "0.5", "(9, 11)": "0.5", "(9, 17)": "0.5", "(9, 25)": "0.5", "(10, 21)": "0.5", "(10, 13)": "0.5", "(11, 20)": "0.5", "(12, 14)": "0.5", "(12, 26)": "0.5", "(13, 26)": "0.5", "(13, 16)": "0.5", "(14, 18)": "0.5", "(15, 21)": "0.5", "(15, 22)": "0.5", "(16, 17)": "0.5", "(17, 20)": "0.5", "(18, 24)": "0.5", "(18, 27)": "0.5", "(20, 24)": "0.5", "(22, 27)": "0.5", "(23, 25)": "0.5", "(24, 27)": "0.5"}
d_qubo = {eval(k): v for k, v in d_qubo.items()}

In [108]:
N = 24
G = nx.random_regular_graph(5, N)
quso = qv.QUSO()

for e in G.edges:
    quso[e] = 1

dg = Diagonals.brute_force_qv(qv.utils.quso_to_qubo(quso))
sv_init = Statevector.make_plus_state(N)

In [109]:
def get_expecs(sv: Statevector):
    shape = (2,) * sv.n_qubits
    p_reshape = np.abs(sv.to_numpy().reshape(shape)) ** 2
    probs = [p_reshape.sum(axis=tuple(i for i in range(sv.n_qubits) if i != j))[1] for j in range(sv.n_qubits)]
    return np.array(probs)

ordering = np.argsort(dg.to_numpy())

def get_expecs_biased(sv: Statevector, alpha: float, ordering: np.ndarray):
    probs = np.abs(sv.to_numpy()) ** 2
    invalid = np.cumsum(probs[ordering]) > alpha
    if invalid[0]:
        invalid[0] = False
    probs[ordering[invalid]] = 0
    probs /= np.sum(probs)

    shape = (2,) * sv.n_qubits
    p_reshape = probs.reshape(shape)
    probs = [p_reshape.sum(axis=tuple(i for i in range(sv.n_qubits) if i != j))[1] for j in range(sv.n_qubits)]
    return np.array(probs)

def gen_expecs(exp: np.ndarray):
    res = np.ones(1)
    for e in exp:
        res = np.kron(res, [1 - e, e])
    return Statevector.from_numpy(np.sqrt(res).astype(np.complex128))

In [110]:
sv = sv_init
for _ in range(10):
    exp = get_expecs_biased(sv, 0.01, ordering)
    sv = gen_expecs(exp)
    print(dg.expec(sv))

-0.006029193702138124
-0.09156125052143589
-0.8368786638057151
-5.171401258196373
-16.926384112493263
-26.241212576959327
-34.0


/var/folders/0y/wbcxhvy92xz19jsf8x4s17km0000gn/T/ipykernel_37452/1212652098.py:15: RuntimeWarning: invalid value encountered in divide
  probs /= np.sum(probs)


nan
nan
nan


In [83]:
expecs = [0.1, 0.2, 0.5, 0.7]

get_expecs(gen_expecs(expecs))

array([0.1, 0.2, 0.5, 0.7])

In [ ]:

mpl_draw(rx.r(3, 3))

In [ ]:
g = rx.PyGraph()
g.add_nodes_from(range(28))
for k, v in d_qubo.items():
    g.add_edge(*k, v)

In [ ]:
mpl_draw(g)

In [75]:
dg.min_val

-20.0

In [111]:
dg.min_val

-34.0